In [ ]:
#from google.colab import drive
#from google.colab import files

In [ ]:
#!mkdir GD

In [ ]:
#drive.mount('/GD',force_remount=True)

In [ ]:
#!cp -r /GD/'My Drive'/NewDataFrame.csv .

In [ ]:
# !cp -r /GD/'My Drive'/pipes_.py .

In [ ]:
# the first 5 lines for collab, GPU!!!! For better speed

#In this section: applimentation of transformer(pipes_), separation the target variable, scaling and standardization, division data into test and training. Applimentation of Adaptive boosting (ML with Teacher) with feature_selection module. Usage of Grid search optimization algorithm which lets us select the best parameters for our optimization problem.


#The classes in the sklearn.feature_selection module  used for feature dimensionality reduction on sample sets, either to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets.

In [1]:
import pandas as pd
import pipes_ as pp
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score

# 1.We review the information via transformer. Preparing data for processing format

In [2]:
df_ = pd.read_csv('NewDataFrame.csv')
#df_.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#df_.drop(["a"], axis=1, inplace=True)

In [3]:
df_ = pp.myPipe.transform(df_)

In [4]:
df_.head()

,Month,Age,AnnualIncome,MonthlyInhandSalary,NumBankAccounts,NumCreditCard,InterestRate,NumofLoan,Delayfromduedate,NumofDelayedPayment,...,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,1,2,3,4,5,6,7
0,0.0,23.000000,19114.12,1824.843333,3,4,3,4.0,3,7.000000,...,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,23.000000,19114.12,4194.170850,3,4,3,4.0,-1,13.220499,...,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2.0,33.307829,19114.12,4194.170850,3,4,3,4.0,3,7.000000,...,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3.0,23.000000,19114.12,4194.170850,3,4,3,4.0,5,4.000000,...,1,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4.0,23.000000,19114.12,1824.843333,3,4,3,4.0,6,13.220499,...,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Separation of the target variable

In [5]:
X = df_.drop(['CreditScore','PaymentBehaviour','Occupation'], axis = 1)

In [6]:
Y = df_['CreditScore']

# Создание проверочного набора (Create a validation set)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size = 0.4, random_state = 15)

In [8]:
X_train.shape, Y_train.shape

((60000, 41), (60000,))

# a. ML With Teacher

# Отбор признаков (feature selection)

# WE SHOULD DO IT BEFORE MINMAX SCALER!

In [9]:
from sklearn.feature_selection import SelectKBest,chi2,f_classif
#специальные методики,оценивают, какие признаки хорошо влияют на зависимые переменные,какие плохо и т.д.(хи квадрат используем)
#нужно передать количество признаков,которое мы хотим отобрать и критерий по которому будем отбирать(наш хи-квадрат)

Selection using the chi function

In [10]:
#k - количество признаков
Selector = SelectKBest(f_classif, k=9)
Selector.fit(X, Y)

SelectKBest(k=9)

In [11]:
#покажет признаки,которые он выбирает,что подходит нам и что нет,проверит степень влияния
Selector.get_support()


array([False, False, False,  True, False, False,  True, False,  True,
        True,  True,  True,  True, False,  True, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [12]:
X.columns[Selector.get_support()].tolist()

['MonthlyInhandSalary',
 'InterestRate',
 'Delayfromduedate',
 'NumofDelayedPayment',
 'NumCreditInquiries',
 'CreditMix',
 'OutstandingDebt',
 'PaymentofMinAmount',
 'CreditHistoryYears']

In [13]:
drop_cols = X.columns[~Selector.get_support()].tolist()

In [14]:
drop_cols

['Month',
 'Age',
 'AnnualIncome',
 'NumBankAccounts',
 'NumCreditCard',
 'NumofLoan',
 'CreditUtilizationRatio',
 'TotalEMIpermonth',
 'MonthlyBalance',
 'Occupation_Accountant',
 'Occupation_Architect',
 'Occupation_Developer',
 'Occupation_Doctor',
 'Occupation_Engineer',
 'Occupation_Entrepreneur',
 'Occupation_Journalist',
 'Occupation_Lawyer',
 'Occupation_Manager',
 'Occupation_Mechanic',
 'Occupation_MediaManager',
 'Occupation_Musician',
 'Occupation_NoOccupation',
 'Occupation_Scientist',
 'Occupation_Teacher',
 'Occupation_Writer',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7']

In [15]:
X_new = X.drop(drop_cols, axis = 1)

In [16]:
X_new

,MonthlyInhandSalary,InterestRate,Delayfromduedate,NumofDelayedPayment,NumCreditInquiries,CreditMix,OutstandingDebt,PaymentofMinAmount,CreditHistoryYears
0,1824.843333,3,3,7.000000,4.0,3,809.98,1,22.000000
1,4194.170850,3,-1,13.220499,4.0,1,809.98,1,17.966516
2,4194.170850,3,3,7.000000,4.0,1,809.98,1,22.000000
3,4194.170850,3,5,4.000000,4.0,1,809.98,1,22.000000
4,1824.843333,3,6,13.220499,4.0,1,809.98,1,22.000000
...,...,...,...,...,...,...,...,...,...
99995,3359.415833,7,23,7.000000,3.0,3,502.38,1,31.000000
99996,3359.415833,7,18,7.000000,3.0,3,502.38,1,31.000000
99997,3359.415833,0,27,6.000000,3.0,1,502.38,1,31.000000
99998,3359.415833,7,20,13.220499,3.0,1,502.38,1,31.000000


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train,  Y_test = train_test_split(X_new, Y, test_size = 0.5, random_state = 15)

In [18]:
X_train.shape

(50000, 9)

# scaling and standardization of values

Standardize features by removing the mean and scaling to unit variance.
Standardization of a dataset is a common requirement for many machine learning estimators: 
they might behave badly if the individual features do not more or less look like standard normally distributed data.

Method fit_transform:Fits transformer to X and y with optional parameters fit_params and returns a transformed version of X.

In [19]:
#from sklearn.preprocessing import StandardScaler
#mms = StandardScaler()
#X = mms.fit_transform(X)

#from sklearn.preprocessing import MinMaxScaler
#mms = MinMaxScaler()
#X = mms.fit_transform(X_train)

# Адаптивный бустинг (Adaptive boosting)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
#clf = tree.DecisionTreeClassifier()
from sklearn.tree import DecisionTreeClassifier  ### Замена класса
from sklearn.ensemble import RandomForestClassifier

In [21]:
Ada_clf = AdaBoostClassifier()

In [22]:
#Ada_clf = AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(max_depth = 7))
paramgrid = [{'base_estimator': [DecisionTreeClassifier(max_depth = 5)], 
              'learning_rate' : [0.1,0.5,1], 'n_estimators': [10,100,200]}, 
             
             
             {'base_estimator': [RandomForestClassifier(max_depth = 2, n_estimators = 10)], 
              'learning_rate' : [0.1,1],
              'n_estimators': [10]}
            ]

In [23]:
GS_ada = GridSearchCV(Ada_clf, param_grid = paramgrid, cv = 3,scoring = 'f1_macro')

In [24]:
GS_ada.fit(X_train, Y_train)

GridSearchCV(cv=3, estimator=AdaBoostClassifier(),
             param_grid=[{'base_estimator': [DecisionTreeClassifier(max_depth=5)],
                          'learning_rate': [0.1, 0.5, 1],
                          'n_estimators': [10, 100, 200]},
                         {'base_estimator': [RandomForestClassifier(max_depth=2,
                                                                    n_estimators=10)],
                          'learning_rate': [0.1, 1], 'n_estimators': [10]}],
             scoring='f1_macro')

In [26]:
GS_ada.best_params_

{'base_estimator': DecisionTreeClassifier(max_depth=5),
 'learning_rate': 0.1,
 'n_estimators': 100}

In [30]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),learning_rate = 0.1, n_estimators = 100)
ada.fit(X_train,Y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.1, n_estimators=100)

In [32]:
from sklearn.metrics import f1_score
pred = ada.predict(X_test)
f1_score(Y_test,pred,average='macro')

0.6622632595859613

In [33]:
from sklearn.model_selection import cross_val_score, cross_val_predict

In [35]:
cs = cross_val_score(GS_ada,X_train, Y_train, cv= 5, scoring = 'f1_macro')

In [36]:
cs

array([0.6585128 , 0.66145375, 0.66762392, 0.66964992, 0.66198824])

In [37]:
ada.score(X_test,Y_test), ada.score(X_train, Y_train)

(0.68216, 0.7005)

In [38]:
pred = ada.predict(X_test)

In [39]:
pred

array([0, 0, 2, ..., 0, 0, 0], dtype=int64)

In [40]:
confusion_matrix(Y_test,pred)


array([[19395,  4311,  2886],
       [ 2117,  6545,   174],
       [ 5167,  1237,  8168]], dtype=int64)

In [41]:
precision_score(Y_test,pred, average='macro'), recall_score(Y_test,pred, average='macro'), f1_score(Y_test,pred, average='macro') 

(0.6652218382668756, 0.6768671713344226, 0.6622632595859613)

In [ ]:
import pickle
with open('data.pickle', 'wb') as f:
     pickle.dump(ada, f)

In [ ]:
with open('data.pickle', 'rb') as f:
    new_model = pickle.load(f)